In [ ]:
import os
import torch
import torch.nn as nn
import shap
import numpy as np
import matplotlib.pyplot as plt
from torchvision import transforms
from PIL import Image

os.environ["CUDA_VISIBLE_DEVICES"] = "2"
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

root_dir = "../"
data_dir = os.path.join(root_dir, "data")
model_dir = os.path.join(root_dir, "pretrained_model")
explanation_dir = os.path.join(root_dir, "explanations")

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

model = torch.hub.load("pytorch/vision:v0.10.0", "resnet18", pretrained=False)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)
model = model.to(device)
model.load_state_dict(
    torch.load(os.path.join(model_dir, "model.pt"), map_location=device)
)
model.to(device)
model.eval()
x = torch.randn(1, 3, 1200, 1600, device=device)
model(x)
torch.cuda.empty_cache()

mean = torch.tensor([0.485, 0.456, 0.406])
std = torch.tensor([0.229, 0.224, 0.225])
transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize(mean=mean, std=std)]
)

print(f"Running shap version {shap.__version__}")

In [ ]:
ref = torch.load(os.path.join(explanation_dir, "reference.pt"))
ref = ref.permute(1, 2, 0)
ref = ref.numpy()
masker = shap.maskers.Image(ref)


def f(x):
    x = torch.tensor(x).float()
    x = x.permute(0, 3, 1, 2)
    x = x.to(device)
    return model(x).detach().cpu().numpy()[..., -1]


partexp = shap.Explainer(f, masker)

In [ ]:
true_positives = np.load(os.path.join(explanation_dir, "true_positive.npy"))

for i, image_path in enumerate(true_positives[[0]]):
    image_name = os.path.basename(image_path).split(".")[0]
    image = Image.open(os.path.join(root_dir, image_path))
    image_t = transform(image)
    image_np = image_t.permute(1, 2, 0).numpy()
    explanation = partexp(
        np.expand_dims(image_np, axis=0), max_evals=500, fixed_context=0
    )

    explanation = explanation.values[0].sum(axis=-1)
    _max = np.amax(np.abs(explanation.flatten()))
    plt.imshow(image)
    plt.imshow(explanation, cmap="bwr", alpha=0.5, vmin=-_max, vmax=_max)
    plt.show()
    plt.imshow(image)
    plt.imshow(explanation > 0, cmap="bwr", alpha=0.5)
    plt.show()
    break